In [1]:
# 参考 https://colab.research.google.com/github/TadaoYamaoka/ShogiAIBook2/blob/main/notebook/train.ipynb#scrollTo=zAT8jrKysnZS
# dlshogiの評価関数モデルをCore ML形式に変換
# バッチサイズ1の最もシンプルなモデル

In [2]:
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Tokyo /etc/localtime
!date

Sat Jan 22 19:06:35 JST 2022


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls -l "/content/drive/My Drive/ShogiAIBookData/"

total 9932211
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-001.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-002.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-003.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-004.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-005.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-006.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-007.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-008.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-009.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-010.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-011.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-012.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-013.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlsh

In [5]:
!pip install dlshogi==0.1.1

     |████████████████████████████████| 3.3 MB 13.2 MB/s 


In [6]:
!pip install coremltools

     |████████████████████████████████| 1.6 MB 14.0 MB/s 


In [7]:
%cd /content/ShogiAIBook

[Errno 2] No such file or directory: '/content/ShogiAIBook'
/content


In [8]:
!ln -s "/content/drive/My Drive/ShogiAIBook/log"   ./log
!ln -s "/content/drive/My Drive/ShogiAIBook/model" ./model
!ln -s "/content/drive/My Drive/ShogiAIBookData"   ./data

In [9]:
!ls -ld ./data/*

-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-001.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-002.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-003.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-004.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-005.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-006.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-007.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-008.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-009.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-010.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-011.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-012.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data

In [10]:
!cp ./data/model/checkpoint_resnet10_swish-072.pth ./model/
!cp ./data/model/model_resnet10_swish-072 ./model/

In [11]:
!ls -l ./model/

total 188968
-rw------- 1 root root 88334907 Jan 22 19:07 checkpoint_resnet10_swish-072.pth
-rw------- 1 root root 29376703 Jan 22 17:50 DlShogiResnet10Swish.mlmodel
-rw------- 1 root root 27347240 Jan 22 19:07 model_resnet10_swish-072
-rw------- 1 root root 48443392 Jan 22 18:46 SampleIO.bin


In [20]:
import torch
from dlshogi.common import *
from dlshogi.network.policy_value_network import policy_value_network
from dlshogi import serializers
from dlshogi import cppshogi

In [13]:
device = "cpu"

In [14]:
model = policy_value_network("resnet10_swish", add_sigmoid=True)

In [15]:
model.set_swish(False)#swishをx*sigmoid(x)で計算するモード

In [16]:
serializers.load_npz("./model/model_resnet10_swish-072", model, False)

In [17]:
model.eval()

PolicyValueNetworkAddSigmoid(
  (l1_1_1): Conv2d(62, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l1_1_2): Conv2d(62, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (l1_2): Conv2d(57, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (l2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l10): Conv2d(192, 192, k

In [21]:
def mini_batch(hcpevec):
    features1 = np.empty((len(hcpevec), FEATURES1_NUM, 9, 9), dtype=np.float32)
    features2 = np.empty((len(hcpevec), FEATURES2_NUM, 9, 9), dtype=np.float32)
    move = np.empty((len(hcpevec)), dtype=np.int64)
    result = np.empty((len(hcpevec)), dtype=np.float32)
    value = np.empty((len(hcpevec)), dtype=np.float32)

    cppshogi.hcpe_decode_with_value(hcpevec, features1, features2, move, result, value)

    z = result.astype(np.float32) - value + 0.5

    return (torch.tensor(features1).to(device),
            torch.tensor(features2).to(device),
            torch.tensor(move.astype(np.int64)).to(device),
            torch.tensor(result.reshape((len(hcpevec), 1))).to(device),
            torch.tensor(z).to(device),
            torch.tensor(value.reshape((len(value), 1))).to(device)
            )

In [22]:
batchsize = 1
hcpevec = np.array([([ 88, 164,  73,  33,  12, 215,  87,  33, 126, 142,  77,  33,  44, 175,  66, 120,  20, 194, 171,  16, 158,  77,  33,  44, 215,  95,  33,  62, 142,  73,  33,  12], 0, 7739, 1, 0)] * batchsize, HuffmanCodedPosAndEval)
x1, x2, t1, t2, z, value = mini_batch(hcpevec)

In [23]:
traced_model = torch.jit.trace(model, (x1, x2))

In [24]:
traced_model

PolicyValueNetworkAddSigmoid(
  original_name=PolicyValueNetworkAddSigmoid
  (l1_1_1): Conv2d(original_name=Conv2d)
  (l1_1_2): Conv2d(original_name=Conv2d)
  (l1_2): Conv2d(original_name=Conv2d)
  (l2): Conv2d(original_name=Conv2d)
  (l3): Conv2d(original_name=Conv2d)
  (l4): Conv2d(original_name=Conv2d)
  (l5): Conv2d(original_name=Conv2d)
  (l6): Conv2d(original_name=Conv2d)
  (l7): Conv2d(original_name=Conv2d)
  (l8): Conv2d(original_name=Conv2d)
  (l9): Conv2d(original_name=Conv2d)
  (l10): Conv2d(original_name=Conv2d)
  (l11): Conv2d(original_name=Conv2d)
  (l12): Conv2d(original_name=Conv2d)
  (l13): Conv2d(original_name=Conv2d)
  (l14): Conv2d(original_name=Conv2d)
  (l15): Conv2d(original_name=Conv2d)
  (l16): Conv2d(original_name=Conv2d)
  (l17): Conv2d(original_name=Conv2d)
  (l18): Conv2d(original_name=Conv2d)
  (l19): Conv2d(original_name=Conv2d)
  (l20): Conv2d(original_name=Conv2d)
  (l21): Conv2d(original_name=Conv2d)
  (l22): Conv2d(original_name=Conv2d)
  (l22_2): Bia

In [25]:
x1.shape

torch.Size([1, 62, 9, 9])

In [26]:
x2.shape

torch.Size([1, 57, 9, 9])

In [27]:
import coremltools as ct

In [28]:
ct_input_x1 = ct.TensorType(name='x1', shape=(1, 62, 9, 9))
ct_input_x2 = ct.TensorType(name='x2', shape=(1, 57, 9, 9))

In [29]:
mlmodel = ct.convert(traced_model, inputs=[ct_input_x1, ct_input_x2])

Running MIL Common passes:   0%|          | 0/34 [00:00<?, ? passes/s]/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '577', of the source model, has been renamed to 'var_577' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '611', of the source model, has been renamed to 'var_611' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 273/273 [00:00<00:00, 287.04 ops/s]


In [30]:
# macでないと実行不可
# results = mlmodel.predict({"x1": x1.numpy(), "x2": x2.numpy()})

In [53]:
# このままだと、出力変数の名前がvar_577のような実装依存の値になり、Swift側での受け取りに不便なので名前を固定する
spec = mlmodel.get_spec()
move_name, result_name = mlmodel.output_description
ct.utils.rename_feature(spec, move_name, "move")
ct.utils.rename_feature(spec, result_name, "result")
mlmodel = ct.models.MLModel(spec)

In [54]:
mlmodel.save("./model/DlShogiResnet10Swish.mlmodel")

In [32]:
move, game_result = model(x1, x2)

In [33]:
move, game_result

(tensor([[-3.1853, -3.2401, -1.9737,  ..., -3.1441, -2.1586, -2.9530]],
        grad_fn=<AddBackward0>),
 tensor([[0.5265]], grad_fn=<SigmoidBackward0>))

In [34]:
move.shape

torch.Size([1, 2187])

In [35]:
game_result.shape

torch.Size([1, 1])